In [30]:
import os
import cv2
import numpy as np

In [31]:
# Set the image path
path = "./"
fileName = "usb-0002.jpg"

In [32]:
# Read the image in default mode:
inputImage = cv2.imread(os.path.join(path, fileName))
# Show current crop:
cv2.imshow("InputImage", inputImage)
cv2.waitKey(0)

-1

In [33]:
# Convert RGB to grayscale:
grayscaleImage = cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)
cv2.imshow("GrayscaleImage", grayscaleImage)
cv2.waitKey(0)

-1

In [34]:
# Set the sample ROI and crop it:
(imageHeight, imageWidth) = grayscaleImage.shape[:2]
imageHeight, imageWidth

(2480, 3507)

In [35]:
# Set the sample ROI:
roiX = 0
roiY = int(0.03 * imageHeight)
roiWidth = imageWidth
roiHeight = int(0.03 * imageHeight)
roiX, roiY, roiWidth, roiHeight

(0, 74, 3507, 74)

In [36]:
# Crop the the sample ROI image:
imageRoi = grayscaleImage[roiY:roiY+roiHeight, roiX:roiWidth]
cv2.imshow("ImageRoi", imageRoi)
cv2.waitKey(0)

-1

In [37]:
# Thresholding:
_, binaryImage = cv2.threshold(imageRoi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow("ImageRoi", binaryImage)
cv2.waitKey(0)

-1

In [38]:
# Reduce the ROI to a 1 x imageWidth row:
reducedImg = cv2.reduce(binaryImage, 0, cv2.REDUCE_MAX)
cv2.imshow("ReducedImg", reducedImg)
cv2.waitKey(0)

-1

In [39]:
reducedImg.shape

(1, 3507)

In [40]:
# Store the transition positions here:
linePositions = []

# Find transitions from 0 to 255:
pastPixel = 255
for x in range(reducedImg.shape[1]):
    # Get current pixel:
    currentPixel = reducedImg[0, x]
    # Check for the "jumps":
    if currentPixel == 255 and pastPixel == 0:
        # Store the jump locations in list:
        print("Got Jump at:"+str(x))
        linePositions.append(x)
    # Set current pixel to past pixel:
    pastPixel = currentPixel

Got Jump at:1729


In [41]:
linePositions

[1729]

In [42]:
reducedImg.shape[1]

3507

In [43]:
imageHeight

2480

In [44]:
linePositions.insert(0, 0)

In [45]:
linePositions

[0, 1729]

In [46]:
maxWidth = reducedImg.shape[1]
closestMiddle = maxWidth // 2
maxWidth, closestMiddle

(3507, 1753)

In [47]:
# Crop pages:
if len(linePositions) == 1:
    linePositions.insert(0, 0)
elif len(linePositions) > 2:
    maxWidth = reducedImg.shape[1]
    halfWidth = maxWidth // 2
    closestMiddle = maxWidth
    for i in range(len(linePositions)):
        if abs(halfWidth - i) < closestMiddle and (abs(halfWidth - i) > (halfWidth // 2)) and ((abs(halfWidth - i) < ((halfWidth // 2) * 3))):
            closestMiddle = i
    if abs(closestMiddle - halfWidth) > 50:
        closestMiddle = halfWidth
    linePositions = [0, closestMiddle]
for i in range(len(linePositions)):
    # Get top left:
    cropX = linePositions[i]

    # Get top left:
    if i != len(linePositions)-1:
        # Get point from the list:
        cropWidth = linePositions[i+1]
    else:
        # Set point from the image's original width:
        cropWidth = reducedImg.shape[1]

    # Crop page:
    cropY = 0
    cropHeight = imageHeight
    currentCrop = inputImage[cropY:cropHeight, cropX:cropWidth]

    # Show current crop:
    #cv2.imshow("CurrentCrop", currentCrop)
    cv2.imwrite(f'CurrentCrop_{i}.jpg', currentCrop)
    #cv2.waitKey(0)